convert .mp3 to .wav
#terminal commands

for f in *.mp3; do ffmpeg -i "$f" -acodec pcm_s16le -ac 1 -ar 16000 "${f%.mps}.wav"; done
mkdir clips_wav
find . -name '*.wav' | xargs mv --target-directory=clips_wav

<!-- -ac 1 converts to mono channel -->
<!-- -ar 16000 sets sampling rate to 16kHz -->

In [24]:
import itertools
import pandas as pd
import re
import subprocess
from collections import Counter

In [6]:
train = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/train.tsv', sep='\t')
dev = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/dev.tsv', sep='\t')
test = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/test.tsv', sep='\t')
validated = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/validated.tsv', sep='\t')

/tmp/ipykernel_90333/2965464420.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  validated = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/validated.tsv', sep='\t')


In [29]:
# train = pd.read_csv('sampla_data/train.csv')
# train
train = pd.read_csv('/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/train.tsv', sep='\t')
train.to_csv('preparation_scripts/experiment_data/train_9.csv')

In [26]:
Counter(list(train['sentence'])).most_common()

[('mikumi inajaribu kulinganishwa mara kwa mara na tambarare maarufu za serengeti',
  12),
 ('wazaramo ambao kiasili watani zao ni wanyamwezi', 11),
 ('chunga afya yako', 11),
 ('hussein ali mwinyi ni mwana wa rais wa zamani ali hassan mwinyi', 11),
 ('alikuwa mwanamke wa kwanza kuchaguliwa na kuongoza nchi ya kiislamu', 11),
 ('mwaka elfu moja mia tisa tisini na sita', 11),
 ('maeneo mengi yana wastani wa kina cha mita arobaini', 11),
 ('wanawake huimba nyimbo tulivu na nyimbo za kuwasifu watoto wao', 11),
 ('waliotumia kuanzia kubalehe hadi kumzaa mtoto wa kwanza', 11),
 ('jina wasukuma lina maana ya watu wa upande wa kaskazini', 10),
 ('kufuatana na uainishaji wa lugha kwa ndani zaidi kibakatisara iko katika kundi la kilanddayak',
  10),
 ('tume hii ilianza na kazi ya kutunga katiba ya chama cha mapinduzi', 10),
 ('kwashakoo ni ugonjwa utokanao na ukosefu wa viini vya protini mwilini', 10),
 ('urembo ni hali ya kuwa na afya bora', 10),
 ('tumbili wajulikanao kama mbega weusi na weup

In [7]:
duplicates = {}

# dictionary with ids of duplicate instances grouped together
#for (a,b) in sent_sets:
#    duplicates[a] = duplicates[a] + [b]

#loop through every unique sentence in the DataFrame
for sentence in train['sentence'].unique():

    # get the indices of the rows where this sentence occurs
    indices = train.index[train['sentence'] == sentence].tolist()

    # if there are more than one row with this sentence, add it to the dictionary
    duplicates[sentence] = indices


In [22]:
# we create the gold, silver and bronze standard by including the first, second and third instance of each item 
one2one = []
one2two = []
one2four = []
one2eight = []
one2eleven = []

for dups in duplicates.values():
    one2one.append(dups[0:1])
    one2two.append(dups[0:2])
    one2four.append(dups[0:4])
    one2eight.append(dups[0:8])
    one2eleven.append(dups[0:11])

In [23]:
one2two = [item for sublist in one2two for item in sublist]
train.loc[one2two]
# one2two

,Unnamed: 0,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,1,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,common_voice_sw_30317714.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,1,NaN,NaN,NaN,NaN,sw,NaN
87519,151117,daba44cc732d258697d78d9c448abb73b2714b6f5149a5...,common_voice_sw_30132287.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,0,NaN,NaN,NaN,NaN,sw,NaN
1,3,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,common_voice_sw_29002392.mp3,inajulikana kama shina la warangi,2,0,NaN,NaN,NaN,NaN,sw,NaN
8670,15625,51d623b701196bc4c289965e219c49c2b9a16943710b55...,common_voice_sw_29238631.mp3,inajulikana kama shina la warangi,2,0,twenties,female,NaN,NaN,sw,NaN
2,5,0627e3da13c92a346dc119f4939b6d84d4712093e75656...,common_voice_sw_37214113.mp3,alikamatwa katika wilaya ya mkono kwa sasa,7,2,twenties,NaN,NaN,NaN,sw,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
131214,224381,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30325157.mp3,raia walipolalamikia ugumu wa upatikanaji wa f...,2,0,thirties,male,NaN,NaN,sw,NaN
131222,224394,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30325205.mp3,aliyewaongoza wajerumani hans meyer na ludwig ...,4,1,thirties,male,NaN,NaN,sw,NaN
131253,224442,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30325391.mp3,diego maradona wa argentina alikuwashikika has...,2,0,thirties,male,NaN,NaN,sw,NaN
131288,224500,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30325574.mp3,jina la kwanza ni mwita ambalo asili yake ni u...,2,0,thirties,male,NaN,NaN,sw,NaN


Identifying sentencces in the training sets that have repetition, for development of our gold, silver and bronze standard dataset settings

In [7]:
# finding the instances from the training set that have been validated
sent_sets = []
for idx, value in train['sentence'].items():
    for idx2, value2 in validated['sentence'].items():
        if value == value2:
            sent_sets.append((idx,idx2))

In [8]:
# dictionary where we will store duplicates
duplicates = dict.fromkeys(train.index, [])

In [9]:
# dictionary with ids of duplicate instances grouped together
for (a,b) in sent_sets:
    duplicates[a] = duplicates[a] + [b]

In [10]:
# we create the gold, silver and bronze standard by including the first, second and third instance of each item 
gold = []
silver = []
bronze = []

for dups in duplicates.values():
    gold.append(dups[0:1])
    silver.append(dups[0:2])
    bronze.append(dups[0:3])

In [11]:
gold = [item for sublist in gold for item in sublist]
silver = [item for sublist in silver for item in sublist]
bronze = [item for sublist in bronze for item in sublist]

In [25]:
validated.iloc[gold].to_csv('gold_training.csv', index=True)
validated.iloc[silver].to_csv('silver_training.csv', index=True)
validated.iloc[bronze].to_csv('bronze_training.csv', index=True)

In [20]:
audio_sizes = pd.DataFrame(size)
audio_sizes = audio_sizes.set_index(0)

In [38]:
validated.iloc[gold].to_csv('gold_training.csv', index=True)

IndexError: DataFrame indexer is not allowed for .iloc
Consider using .loc for automatic alignment.

In [39]:
gold = pd.read_csv('gold_training.csv', index_col=0)
silver = pd.read_csv('silver_training.csv')
bronze = pd.read_csv('bronze_training.csv')

In [54]:
def find_audio_size(training_dataframe):
    
#   Find the size, in bits, of the .wav audio files using sox
    regex = re.compile(r'(File Size *:) ([\d|\.]*)')
    size = []
    for index, row in training_dataframe.iterrows():
        prop = subprocess.run(["soxi", "/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/" + training_dataframe.loc[index]['path'][:-4] + ".wav"], text=True, capture_output=True)
        size.append([index,regex.search(str(prop)).group(2)])
        
    audio_sizes = pd.DataFrame(size)
    audio_sizes = audio_sizes.set_index(0)
    
#   Add the size column to the dataframe  
    output_df = pd.concat([training_dataframe,audio_sizes], axis=1)
    output_df['path'] = output_df['path'].str[:-4] + '.wav'
    output_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + output_df['path']
#   take only the columns needed for training with coqui ai STT toolkit  
    output_df = output_df[['path', 1, 'sentence']]
    output_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
    output_df['wav_filesize'] = output_df['wav_filesize'].astype(float).astype(int)
    output_df.to_csv(training_dataframe + 'for_coqui.csv', index=False, line_terminator='\n')

In [55]:
find_audio_size(gold)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U13')) -> None

In [47]:
for index, row in gold.iterrows():
    print(index)

21215
21216
21217
21218
21219
21220
21221
21222
21223
21226
21227
21228
21229
21230
21232
21233
21234
21235
21244
21245
21247
21248
21249
21250
21252
21253
21254
21255
21256
21257
21259
21260
21261
21262
21263
21264
21265
21268
21269
21270
21271
21273
21274
21275
21276
21277
21278
21279
21280
21283
21286
21287
21288
21289
21290
21291
21292
21293
21294
21295
21297
21298
21299
21300
21302
21303
21305
21306
21309
21310
21311
21312
21313
21314
21316
21318
21319
21320
21321
21322
21323
21326
21329
21331
21333
21334
21338
21339
21340
21341
21343
21344
21345
21346
21347
21349
21352
21353
21354
21355
21356
21358
21360
21361
21362
21363
21364
21365
21367
21370
21372
21374
21376
21377
21378
21380
21381
21382
21383
21384
21386
21387
21391
21394
21395
21396
21397
21398
21399
21400
21401
21402
21403
21404
21405
21406
21407
21408
21410
21412
21413
21414
21415
21416
21417
21419
21421
21422
21423
21425
21426
21427
21428
21430
21431
21432
21434
21435
21437
21440
21441
21443
21444
21448
21449
21450
2145

31429
31434
31436
31437
31439
31441
31450
31451
31452
31453
31454
31455
31456
31463
31464
31465
31466
31467
31469
31472
31474
31476
31477
31479
31481
31484
31485
31486
31487
31488
31490
31491
31493
31497
31498
31501
31504
31505
31508
31509
31510
31511
31512
31514
31515
31516
31517
31518
31521
31523
31524
31525
31531
31532
31534
31540
31546
31548
31549
31550
31551
31552
31555
31558
31562
31563
31564
31565
31566
31569
31571
31573
31574
31575
31578
31579
31580
31581
31585
31586
31592
31596
31597
31598
31601
31602
31603
31604
31605
31609
31610
31612
31613
31615
31616
31618
31620
31621
31625
31626
31627
31629
31630
31633
31634
31636
31639
31640
31641
31645
31646
31647
31648
31649
31650
31651
31652
31653
31654
31657
31658
31659
31660
31662
31663
31665
31666
31668
31670
31674
31675
31676
31679
31687
31689
31690
31691
31693
31696
31697
31698
31699
31701
31702
31704
31707
31709
31710
31712
31714
31715
31717
31718
31721
31722
31723
31724
31725
31726
31728
31729
31730
31735
31736
31737
31738
3173

46726
46727
46728
46730
46732
46733
46738
46739
46740
46742
46744
46745
46747
46752
46756
46761
46762
46763
46765
46767
46768
46770
46771
46773
46779
46790
46791
46793
46796
46797
46798
46805
46808
46811
46818
46821
46822
46823
46824
46827
46829
46830
46833
46835
46836
46837
46841
46842
46845
46848
46853
46859
46860
46862
46866
46868
46869
46875
46881
46883
46888
46889
46891
46892
46894
46896
46906
46915
46916
46919
46921
46923
46926
46937
46941
46945
46947
46949
46950
46953
46954
46955
46958
46960
46961
46962
46964
46967
46970
46977
46978
46982
46983
46985
46986
46988
46989
46990
46991
46992
46993
46996
47001
47002
47005
47007
47014
47017
47019
47020
47021
47024
47026
47027
47028
47031
47032
47037
47040
47042
47043
47045
47047
47049
47058
47059
47065
47066
47067
47068
47073
47074
47075
47077
47079
47080
47081
47082
47083
47087
47088
47092
47094
47097
47099
47100
47104
47111
47114
47115
47121
47122
47128
47135
47136
47138
47139
47142
47144
47145
47147
47150
47155
47156
47158
47160
4718

76368
76419
76523
76526
76527
76533
76538
76555
76574
76576
76579
76584
76586
76610
76633
76636
76641
76657
76675
76681
76714
76745
76763
76782
76793
76796
76800
76801
76805
76809
76812
76813
76817
76833
76834
76840
76848
76850
76852
76853
76855
76857
76869
76881
76886
76888
76895
76897
76899
76908
76915
76916
76935
76936
76943
76946
76951
76968
76969
76981
76982
76985
76987
77035
77038
77057
77096
77123
77158
77189
77207
77233
77257
77270
77273
77280
77369
77385
77391
77408
77414
77433
77472
77698
77849
77936
77961
77965
77966
77980
77981
77983
77984
77992
77993
78002
78004
78015
78024
78026
78028
78032
78036
78040
78044
78047
78056
78066
78069
78070
78087
78093
78096
78098
78101
78102
78105
78107
78123
78132
78139
78147
78148
78149
78151
78152
78157
78165
78171
78176
78178
78180
78181
78184
78187
78188
78194
78198
78199
78203
78206
78207
78209
78216
78220
78231
78233
78234
78236
78244
78246
78290
78302
78394
78424
78435
78442
78547
78617
78653
78700
78710
78712
78758
78768
78778
7880

In [19]:
regex = re.compile(r'(File Size *:) ([\d|\.]*)')
size = []
for index in bronze:
    prop = subprocess.run(["soxi", "/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/" + validated.iloc[index]['path'][:-4] + ".wav"], text=True, capture_output=True)
    size.append([index,regex.search(str(prop)).group(2)])

In [10]:
audio_sizes = pd.DataFrame(size)
audio_sizes = audio_sizes.set_index(0)

In [105]:
bronze_df = pd.concat([validated.loc[bronze],audio_sizes], axis=1)
bronze_df['path'] = bronze_df['path'].str[:-4] + '.wav'
bronze_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + bronze_df['path']
bronze_df = bronze_df[['path', 1, 'sentence']]
bronze_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
bronze_df['wav_filesize'] = bronze_df['wav_filesize'].astype(float).astype(int)

In [114]:
bronze_df.to_csv('bronze_standard.csv', index=False, line_terminator='\n')

In [103]:
silver_df = pd.concat([validated.loc[silver],audio_sizes.loc[silver]], axis=1)
silver_df['path'] = silver_df['path'].str[:-4] + '.wav'
silver_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + silver_df['path']
silver_df = silver_df[['path', 1, 'sentence']]
silver_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
silver_df['wav_filesize'] = silver_df['wav_filesize'].astype(float).astype(int)

In [112]:
silver_df.to_csv('silver_standard.csv', index=False)

In [101]:
gold_df = pd.concat([validated.loc[gold],audio_sizes.loc[gold]], axis=1)
gold_df['path'] = gold_df['path'].str[:-4] + '.wav'
gold_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + gold_df['path']
gold_df = gold_df[['path', 1, 'sentence']]
gold_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
gold_df['wav_filesize'] = gold_df['wav_filesize'].astype(float).astype(int)

In [111]:
gold_df.to_csv('gold_standard.csv', index=False)

data prep for dev and test sets

In [121]:
regex = re.compile(r'(File Size *:) ([\d|\.]*)')
size = []
for file_name in test['path']:
#     print(file_name[:-4])
    prop = subprocess.run(["soxi", "/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/" + file_name[:-4] + ".wav"], text=True, capture_output=True)
    size.append(regex.search(str(prop)).group(2))

In [122]:
sizes = pd.DataFrame(size)
test_df = pd.concat([test,sizes], axis=1)
test_df['path'] = test_df['path'].str[:-4] + '.wav'
test_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + test_df['path']
test_df = test_df[['path', 0, 'sentence']]
test_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
test_df['wav_filesize'] = test_df['wav_filesize'].astype(float).astype(int)

In [124]:
test_df.to_csv('test_df.csv', index=False)

In [39]:
regex = re.compile(r'(File Size *:) ([\d|\.]*)')
size = []
for file_name in dev['path']:
    prop = subprocess.run(["soxi", "/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/" + file_name + ".wav"], text=True, capture_output=True)
    size.append(regex.search(str(prop)).group(2))

In [96]:
sizes = pd.DataFrame(size)
dev_df = pd.concat([dev,sizes], axis=1)
dev_df['path'] = dev_df['path'].str[:-4] + '.wav'
dev_df['path'] = '/media/ks/Elements/cv-corpus-9.0-2022-04-27/sw/clips/clips_wav/' + dev_df['path']
dev_df = dev_df[['path', 0, 'sentence']]
dev_df.columns = ['wav_filename', 'wav_filesize', 'transcript']
dev_df['wav_filesize'] = dev_df['wav_filesize'].astype(float).astype(int)

In [109]:
dev_df.to_csv('dev_df.csv', index=False)

rough work below...

In [145]:
# create list of all mp3 files in bronze for conversion to .wav
' '.join(validated.loc[bronze]['path'].tolist())

'common_voice_sw_28704519.mp3 common_voice_sw_28704530.mp3 common_voice_sw_28812149.mp3 common_voice_sw_28704537.mp3 common_voice_sw_28815583.mp3 common_voice_sw_28704542.mp3 common_voice_sw_28817743.mp3 common_voice_sw_28704543.mp3 common_voice_sw_28815944.mp3 common_voice_sw_28704552.mp3 common_voice_sw_28812161.mp3 common_voice_sw_28704555.mp3 common_voice_sw_28704556.mp3 common_voice_sw_28875936.mp3 common_voice_sw_28704557.mp3 common_voice_sw_28704586.mp3 common_voice_sw_28815498.mp3 common_voice_sw_28704588.mp3 common_voice_sw_28704591.mp3 common_voice_sw_28813395.mp3 common_voice_sw_28704593.mp3 common_voice_sw_28704689.mp3 common_voice_sw_28815786.mp3 common_voice_sw_28704691.mp3 common_voice_sw_28817732.mp3 common_voice_sw_28704692.mp3 common_voice_sw_28814976.mp3 common_voice_sw_28704693.mp3 common_voice_sw_28813284.mp3 common_voice_sw_28704704.mp3 common_voice_sw_29790864.mp3 common_voice_sw_29790865.mp3 common_voice_sw_29790911.mp3 common_voice_sw_29790899.mp3 common_voice_

In [167]:
# create list of all mp3 files in dev and test for conversion to .wav
# ' '.join(dev['path'].tolist())
' '.join(test['path'].tolist())

'common_voice_sw_29002392.mp3 common_voice_sw_31292754.mp3 common_voice_sw_30612341.mp3 common_voice_sw_31291248.mp3 common_voice_sw_31295894.mp3 common_voice_sw_28581699.mp3 common_voice_sw_30667303.mp3 common_voice_sw_30342543.mp3 common_voice_sw_31229192.mp3 common_voice_sw_30644559.mp3 common_voice_sw_30621784.mp3 common_voice_sw_31384593.mp3 common_voice_sw_30644508.mp3 common_voice_sw_30644154.mp3 common_voice_sw_30644782.mp3 common_voice_sw_31414750.mp3 common_voice_sw_28266617.mp3 common_voice_sw_31291434.mp3 common_voice_sw_30577673.mp3 common_voice_sw_31081002.mp3 common_voice_sw_28261304.mp3 common_voice_sw_30562357.mp3 common_voice_sw_30563098.mp3 common_voice_sw_30642167.mp3 common_voice_sw_30644575.mp3 common_voice_sw_30562189.mp3 common_voice_sw_31306454.mp3 common_voice_sw_30620847.mp3 common_voice_sw_30644808.mp3 common_voice_sw_31292777.mp3 common_voice_sw_31152268.mp3 common_voice_sw_30386544.mp3 common_voice_sw_30623659.mp3 common_voice_sw_31255561.mp3 common_voice_